In [ ]:
import numpy as np
import pandas as pd
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

import matplotlib.pyplot as plt

Loading csv data:

In [ ]:
muon_dataframe = pd.read_csv('../MuonPOGAnalysisTemplate/output/bxcut_org.csv')

In [ ]:
def preprocess_features(muon_dataframe):
  """Prepares input features from Muon data set.

  Args:
    muon_dataframe: A Pandas DataFrame expected to contain data
      from muon simulations
  Returns:
    A DataFrame that contains the features to be used for the model.
  """
  selected_features = muon_dataframe[
['Event',
 'n_Primitive',
 '1dtPrimitive.id_r',
 '2dtPrimitive.id_r',
 '3dtPrimitive.id_r',
 '4dtPrimitive.id_r',
 '1dtPrimitive.id_eta',
 '2dtPrimitive.id_eta',
 '3dtPrimitive.id_eta',
 '4dtPrimitive.id_eta',
 '1dtPrimitive.id_phi',
 '2dtPrimitive.id_phi',
 '3dtPrimitive.id_phi',
 '4dtPrimitive.id_phi',
 '1dtPrimitive.phiB',
 '2dtPrimitive.phiB',
 '3dtPrimitive.phiB',
 '4dtPrimitive.phiB',
 '1dtPrimitive.quality',
 '2dtPrimitive.quality',
 '3dtPrimitive.quality',
 '4dtPrimitive.quality',
 'delta_phi12',
 'delta_phi13',
 'delta_phi14',
 'delta_phi23',
 'delta_phi24',
 'delta_phi34']]
  processed_features = selected_features.copy()
  return processed_features.astype(np.float32)



In [ ]:
def preprocess_targets(muon_dataframe):
  """
  Prepares target features (i.e., labels) from muon data set.

  Args:
    muon_dataframe: A Pandas DataFrame expected to contain data
      from the Muon data set.
  Returns:
    A DataFrame that contains the target feature.
  """
  output_targets = pd.DataFrame()
  output_targets["genParticle.pt"] = muon_dataframe["genParticle.pt"]
  return output_targets.astype(np.float32)

In [ ]:
X = preprocess_features(muon_dataframe)

In [ ]:
Y = preprocess_targets(muon_dataframe)

In [ ]:
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=28, kernel_initializer='uniform', activation='relu'))
    model.add(Dropout(rate=0.1))
    model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adagrad')
    return model

In [ ]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=200, verbose=0)

In [ ]:
kfold = KFold(n_splits=8, random_state=seed)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (np.sqrt((-1)*results.mean()), np.sqrt(results.std())))

In [ ]:
predicted = cross_val_predict(estimator, X, Y, cv=kfold)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(Y, predicted, edgecolors=(0, 0, 0))
ax.plot([Y.min(), Y.max()], [Y.min(), Y.max()], 'k--', lw=4)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')

In [ ]:
plt.show()